You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [42]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [43]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [44]:
pip install utils


Note: you may need to restart the kernel to use updated packages.


In [45]:
import os
#from utils import get_openai_api_key
#openai_api_key = get_openai_api_key()
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = "groq/llama-3.1-8b-instant"

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [46]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [47]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [48]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [49]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [50]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [51]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [52]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [53]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f3079c20-d4c5-4440-9265-877da6c421bc                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Thought: I now can give a great answer




🚀 Crew: crew
└── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: ✅ Completed

...

# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.


# Agent: Editor
## Final Answer: 
**The Future of Intelligence: Understanding Artificial Intelligence and its Impact**

As we embark on a new era of technological advancements, Artificial Intelligence (AI) has become an integral part of our lives. From virtual assistants to self-driving cars, AI continues to transform industries and revolutionize the way we live and work. But what exactly is AI, and how will it shape our future? The answer lies in its ability to learn, adapt, and improve on its own, much like human intelligence. By understanding the complexities of AI, we can begin to harness its full potential and unlock a brighter future for all.

**Introduction to Artificial Intelligence**  

AI refers to a broad range of technologies that enable machines to perform tasks that would typically require human intelligence, such as learning, problem-solving, and decision

🚀 Crew: crew
├── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 6e76a461-5cbd-49e1-89a5-4e7e4bf15d6a
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 16f8dbd9-1493-410b-a6ab-8ff208f73f45
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 6e76a461-5cbd-49e1-89a5-4e7e4bf15d6a
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 16f8dbd9-1493-410b-a6ab-8ff208f73f45
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 16f8dbd9-1493-410b-a6ab-8ff208f73f45                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f3079c20-d4c5-4440-9265-877da6c421bc                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the results of your execution as markdown in the notebook.

In [54]:
print(result)

**The Future of Intelligence: Understanding Artificial Intelligence and its Impact**

As we embark on a new era of technological advancements, Artificial Intelligence (AI) has become an integral part of our lives. From virtual assistants to self-driving cars, AI continues to transform industries and revolutionize the way we live and work. But what exactly is AI, and how will it shape our future? The answer lies in its ability to learn, adapt, and improve on its own, much like human intelligence. By understanding the complexities of AI, we can begin to harness its full potential and unlock a brighter future for all.

**Introduction to Artificial Intelligence**  

AI refers to a broad range of technologies that enable machines to perform tasks that would typically require human intelligence, such as learning, problem-solving, and decision-making. AI systems use complex algorithms and data analysis to identify patterns and make predictions or recommendations. The AI market is expected to 

In [55]:
from IPython.display import Markdown
Markdown(result.raw)

**The Future of Intelligence: Understanding Artificial Intelligence and its Impact**

As we embark on a new era of technological advancements, Artificial Intelligence (AI) has become an integral part of our lives. From virtual assistants to self-driving cars, AI continues to transform industries and revolutionize the way we live and work. But what exactly is AI, and how will it shape our future? The answer lies in its ability to learn, adapt, and improve on its own, much like human intelligence. By understanding the complexities of AI, we can begin to harness its full potential and unlock a brighter future for all.

**Introduction to Artificial Intelligence**  
======================================================

AI refers to a broad range of technologies that enable machines to perform tasks that would typically require human intelligence, such as learning, problem-solving, and decision-making. AI systems use complex algorithms and data analysis to identify patterns and make predictions or recommendations. The AI market is expected to reach $190.6 billion by 2025, growing at a CAGR of 38.1% from 2020 to 2025, according to a report by Grand View Research. This rapid growth has led to a surge in AI adoption across various industries, from healthcare to finance.

Opinion: While AI has shown immense potential in various sectors, its rapid growth and development raise concerns about job displacement and bias in decision-making processes. As AI continues to evolve, it's essential to address these concerns and ensure that the technology benefits humanity as a whole. This requires a collaborative effort from policymakers, industry leaders, and AI developers to create a framework that prioritizes ethics, transparency, and accountability.

**Types of Artificial Intelligence**  
=====================================

There are several types of AI, including:

* **Narrow or Weak AI**: Designed to perform a specific task, such as image recognition or language translation. Narrow AI is a subset of AI that is trained on a narrow range of tasks and can be used in applications like chatbots and virtual assistants.
* **General or Strong AI**: A hypothetical AI system that possesses human-like intelligence and can perform any intellectual task that a human can. General AI is still in its infancy and faces significant challenges in terms of development and safety.
* **Superintelligence**: An AI system that significantly surpasses human intelligence in terms of problem-solving and decision-making capabilities. While superintelligence holds promise, it also raises concerns about the potential risks and consequences of creating an AI system that is beyond human control.

While AI has made tremendous progress in recent years, the development of general and superintelligence AI remains a topic of debate among experts. According to a report by the Future of Humanity Institute, there is a significant risk of developing AI systems that are uncontrollable and pose an existential threat to humanity. As AI continues to advance, it's essential to prioritize ethics and safety in its development and deployment, and to ensure that the benefits of AI are shared by all.

**Applications of Artificial Intelligence**  
==========================================

AI has numerous applications across various industries, including:

* **Healthcare**: AI is being used to develop personalized medicine, detect diseases, and assist in medical diagnosis. AI-powered health chatbots can also provide patients with information and support, helping to improve health outcomes and patient satisfaction.
* **Finance**: AI is being used to detect financial anomalies, prevent cyberattacks, and optimize investment strategies. AI-powered trading systems can also analyze vast amounts of data to make informed investment decisions.
* **Transportation**: AI is being used to develop self-driving cars, optimize traffic routes, and enhance passenger safety. AI-powered driverless vehicles can also reduce accidents and congested roads, improving the overall transportation experience.

As AI continues to transform industries, it's essential to address the challenges associated with its deployment, such as data bias and algorithmic decision-making. By prioritizing ethics and accountability, we can harness the full potential of AI and create a better future for all.

**Conclusion and Future Directions**  
=====================================

In conclusion, AI has the potential to revolutionize various aspects of our lives, from healthcare to transportation. However, its rapid growth and development raise concerns about job displacement, bias, and accountability. As we embark on the path to AI development, it's essential to prioritize ethics and safety, ensuring that the technology benefits humanity as a whole.

Opinion: As AI continues to advance, it's crucial that we develop a shared understanding of its implications and risks. By engaging in open and informed discussions, we can navigate the challenges associated with AI and harness its full potential to create a better future for all. This requires a collaborative effort from policymakers, industry leaders, and AI developers to create a framework that prioritizes ethics, transparency, and accountability.

Sources:

* Grand View Research. (2020). Artificial Intelligence Market Size, Share & Trends Analysis Report.
* Future of Humanity Institute. (2019). Artificial General Intelligence.
* World Economic Forum. (2020). The Future of Jobs Report 2020.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [57]:
topic = "DATA SCIENCE"
result = crew.kickoff(inputs={"topic": topic})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f3079c20-d4c5-4440-9265-877da6c421bc                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on DATA SCIENCE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Thought: I now can give a great answer

**Comprehensive Content Plan Document: "Unlocking the Power of Data Science"**

**I. Executive Summary**

Our content plan aims to educate readers on the latest trends, key players, and innovations in data science. We will cater to professionals and enthusiasts interested in data analysis, machine learning, and artificial intelligence. This article will serve as a beginner's guide, covering the basics while highlighting the potential of data science in various industries.

**II. Audience Analysis**

* **Demographics:**
 + Age: 25-45
 + Education: Bachelor's deg

🚀 Crew: crew
└── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: ✅ Completed

...

# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.


# Agent: Editor
## Final Answer: 
Final Answer

I have completed reviewing the given blog post for grammatical errors and alignment with the brand's voice. I have ensured that the post follows journalistic best practices, provides balanced viewpoints when providing opinions or assertions, and avoids major controversial topics or opinions when possible. I have also edited the post to align with the writing style of the organization.

Here is the final version of the blog post in markdown format, ready for publication:

# Unlocking the Power of Data Science: A Beginner's Guide

## Introduction

Data science has revolutionized the way we analyze and interpret data, unlocking new insights and possibilities across various industries. From healthcare to finance, and customer service to e-commerce, data science has become an essential tool for understanding and driving busines

🚀 Crew: crew
├── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 6e76a461-5cbd-49e1-89a5-4e7e4bf15d6a
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 16f8dbd9-1493-410b-a6ab-8ff208f73f45
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b35f85bd-e98b-4ba9-b3f1-293037dd16ce
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 6e76a461-5cbd-49e1-89a5-4e7e4bf15d6a
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 16f8dbd9-1493-410b-a6ab-8ff208f73f45
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 16f8dbd9-1493-410b-a6ab-8ff208f73f45                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f3079c20-d4c5-4440-9265-877da6c421bc                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).